In [21]:
import cv2
import numpy as np
import pyautogui
import time
from pywinauto.application import Application
from pywinauto.findwindows import ElementNotFoundError
from PIL import ImageGrab
import re

def get_window_bbox_by_substring(title_substring):
    """Busca ventana que contenga el texto dado en el título. Devuelve bounding box."""
    app = Application().connect(title_re=f".*{re.escape(title_substring)}.*")
    dlg = app.window(title_re=f".*{re.escape(title_substring)}.*")
    dlg.set_focus()
    rect = dlg.rectangle()
    return (rect.left, rect.top, rect.right, rect.bottom)

def find_image_in_window(template_path, window_bbox, threshold=0.8):
    """Busca una imagen dentro de una región de la ventana, devuelve coordenadas absolutas si encuentra."""
    screenshot = ImageGrab.grab(bbox=window_bbox)
    screenshot_cv = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    
    template = cv2.imread(template_path, cv2.IMREAD_COLOR)
    res = cv2.matchTemplate(screenshot_cv, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(res >= threshold)
    
    if len(loc[0]) > 0:
        top_left = (loc[1][0], loc[0][0])
        h, w = template.shape[:2]
        center_rel = (top_left[0] + w // 2, top_left[1] + h // 2)
        center_abs = (window_bbox[0] + center_rel[0], window_bbox[1] + center_rel[1])
        return center_abs
    return None

def wait_for_image_and_send_hotkey_in_window(title_substring, image_path, timeout=30):
    """Activa una ventana, espera a que aparezca una imagen, y luego envía Alt+C."""
    try:
        print(f"Buscando ventana con '{title_substring}' en el título...")
        window_bbox = get_window_bbox_by_substring(title_substring)
    except ElementNotFoundError:
        print(f"No se encontró ninguna ventana con '{title_substring}' en el título.")
        return False

    print("Esperando a que aparezca la imagen...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        pos = find_image_in_window(image_path, window_bbox)
        if pos:
            print(f"Imagen detectada en {pos}, enviando Alt+C")
            pyautogui.hotkey('alt', 'c')  # Simula Alt + C
            return True
        time.sleep(1)
    
    print("Timeout: no se detectó la imagen.")
    return False

def wait_for_image_and_type_text_in_window(title_substring, image_path, text_to_type="A-B-C", timeout=30):
    """Activa una ventana, espera a que aparezca una imagen, y luego escribe texto letra por letra."""
    try:
        print(f"Buscando ventana con '{title_substring}' en el título...")
        window_bbox = get_window_bbox_by_substring(title_substring)
    except ElementNotFoundError:
        print(f"No se encontró ninguna ventana con '{title_substring}' en el título.")
        return False

    print("Esperando a que aparezca la imagen...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        pos = find_image_in_window(image_path, window_bbox)
        if pos:
            print(f"Imagen detectada en {pos}, escribiendo texto: '{text_to_type}'")
            pyautogui.click(pos)  # Opcional: clic donde se detectó la imagen, por si es un campo de texto
            pyautogui.write(text_to_type, interval=0.1)  # Escribe letra por letra
            return True
        time.sleep(1)

    print("Timeout: no se detectó la imagen.")
    return False

def wait_and_click_in_window(title_substring, trigger_img, destination_img, timeout=30):
    """Espera a detectar 'trigger_img' en una ventana que contenga el texto en su título, y hace clic en 'destination_img'."""
    try:
        print(f"Buscando ventana con '{title_substring}' en el título...")
        window_bbox = get_window_bbox_by_substring(title_substring)
    except ElementNotFoundError:
        print(f"No se encontró ninguna ventana con '{title_substring}' en el título!")
        return False
    
    print("Esperando trigger...")
    start_time = time.time()
    while True:
        pos_trigger = find_image_in_window(trigger_img, window_bbox)
        if pos_trigger:
            print(f"Trigger detectado en {pos_trigger}")
            break
        elif time.time() - start_time > timeout:
            print("Timeout esperando trigger!")
            return False
        time.sleep(1)
    
    print("Buscando destino...")
    pos_dest = find_image_in_window(destination_img, window_bbox)
    if pos_dest:
        print(f"Destino encontrado en {pos_dest}, haciendo clic en {destination_img}")
        pyautogui.moveTo(pos_dest)
        pyautogui.click()
        return True
    else:
        print("No se encontró la imagen destino!")
        return False

window_string = "10.73.162.7"
timeout = 1200

wait_and_click_in_window(window_string, "Mini_Optimize_1_button.png", "Mini_Optimize_1_button.png", timeout=timeout)

wait_and_click_in_window(window_string, "End_stage_1.png", "Ok_button.png", timeout=timeout)

wait_and_click_in_window(window_string, "Mini_Optimize_1_button.png", "Mini_Optimize_1_button.png", timeout=timeout)

wait_and_click_in_window(window_string, "Opt_console_button.png", "Opt_console_button.png", timeout=timeout)

wait_and_click_in_window(window_string, "message_filter.png", "message_filter.png", timeout=timeout)

wait_for_image_and_type_text_in_window(window_string, "message_filter.png", text_to_type="checked for 4 times", timeout=timeout)

wait_and_click_in_window(window_string, "End_Stage_2.png", "close_opt_console.png", timeout=timeout)

wait_and_click_in_window(window_string, "Truncate_Stage_2.png", "Truncate_Stage_2.png", timeout=timeout)

wait_and_click_in_window(window_string, "Segmentation_complete.png", "Ok_button.png", timeout=timeout)

wait_and_click_in_window(window_string, "Final_dose_calculation.png", "disquete.png", timeout=timeout)


Buscando ventana con '10.73.162.7' en el título...
Esperando trigger...
Trigger detectado en (163, 10)
Buscando destino...
Destino encontrado en (163, 10), haciendo clic en Mini_Optimize_1_button.png
Buscando ventana con '10.73.162.7' en el título...
Esperando trigger...
Trigger detectado en (954, 571)
Buscando destino...
Destino encontrado en (1044, 632), haciendo clic en Ok_button.png
Buscando ventana con '10.73.162.7' en el título...
Esperando trigger...
Trigger detectado en (163, 10)
Buscando destino...
Destino encontrado en (163, 10), haciendo clic en Mini_Optimize_1_button.png
Buscando ventana con '10.73.162.7' en el título...
Esperando trigger...
Trigger detectado en (220, 10)
Buscando destino...
Destino encontrado en (220, 10), haciendo clic en Opt_console_button.png
Buscando ventana con '10.73.162.7' en el título...
Esperando trigger...
Trigger detectado en (418, 597)
Buscando destino...
Destino encontrado en (418, 597), haciendo clic en message_filter.png
Buscando ventana con

True

In [20]:
import cv2
import numpy as np
import time
from pywinauto.application import Application
from pywinauto.findwindows import ElementNotFoundError
from pywinauto.mouse import move, click
from PIL import ImageGrab
import re

def get_window_and_bbox(title_substring):
    app = Application().connect(title_re=f".*{re.escape(title_substring)}.*")
    dlg = app.window(title_re=f".*{re.escape(title_substring)}.*")
    dlg.set_focus()
    rect = dlg.rectangle()
    return dlg, (rect.left, rect.top, rect.right, rect.bottom)

def find_image_in_window(template_path, window_bbox, threshold=0.8):
    screenshot = ImageGrab.grab(bbox=window_bbox)
    screenshot_cv = cv2.cvtColor(np.array(screenshot), cv2.COLOR_RGB2BGR)
    template = cv2.imread(template_path, cv2.IMREAD_COLOR)
    res = cv2.matchTemplate(screenshot_cv, template, cv2.TM_CCOEFF_NORMED)
    loc = np.where(res >= threshold)
    
    if len(loc[0]) > 0:
        top_left = (loc[1][0], loc[0][0])
        h, w = template.shape[:2]
        center_rel = (top_left[0] + w // 2, top_left[1] + h // 2)
        center_abs = (window_bbox[0] + center_rel[0], window_bbox[1] + center_rel[1])
        return center_abs
    return None

def click_at_position(pos):
    move(coords=pos)
    click(coords=pos)

def wait_and_click(dlg, window_bbox, trigger_img, destination_img, timeout=30):
    print("Esperando trigger...")
    start_time = time.time()
    while True:
        pos_trigger = find_image_in_window(trigger_img, window_bbox)
        if pos_trigger:
            print(f"Trigger detectado en {pos_trigger}")
            break
        elif time.time() - start_time > timeout:
            print("Timeout esperando trigger!")
            return False
        time.sleep(1)

    print("Buscando destino...")
    pos_dest = find_image_in_window(destination_img, window_bbox)
    if pos_dest:
        print(f"Destino encontrado en {pos_dest}, haciendo clic")
        click_at_position(pos_dest)
        return True
    else:
        print("No se encontró imagen destino")
        return False

def wait_for_image_and_send_hotkey(dlg, window_bbox, image_path, timeout=30):
    print("Esperando a que aparezca la imagen...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        pos = find_image_in_window(image_path, window_bbox)
        if pos:
            print("Imagen detectada, enviando Alt+C")
            dlg.type_keys('%C')  # % representa Alt en pywinauto
            return True
        time.sleep(1)
    print("Timeout sin detectar imagen.")
    return False

def wait_for_image_and_type_text(dlg, window_bbox, image_path, text_to_type="A-B-C", timeout=30):
    print("Esperando a que aparezca la imagen...")
    start_time = time.time()
    while time.time() - start_time < timeout:
        pos = find_image_in_window(image_path, window_bbox)
        if pos:
            print(f"Imagen detectada en {pos}, escribiendo '{text_to_type}'")
            click_at_position(pos)
            time.sleep(0.5)
            dlg.type_keys(text_to_type, with_spaces=True, pause=0.1)
            return True
        time.sleep(1)
    print("Timeout sin detectar imagen.")
    return False

# --- Secuencia principal automatizada ---
window_string = "10.73.162.7"
timeout = 1200

try:
    dlg, bbox = get_window_and_bbox(window_string)

    wait_and_click(dlg, bbox, "Mini_Optimize_1_button.png", "Mini_Optimize_1_button.png", timeout)
    wait_and_click(dlg, bbox, "End_stage_1.png", "Ok_button.png", timeout)
    wait_and_click(dlg, bbox, "Mini_Optimize_1_button.png", "Mini_Optimize_1_button.png", timeout)
    wait_and_click(dlg, bbox, "Opt_console_button.png", "Opt_console_button.png", timeout)
    wait_and_click(dlg, bbox, "message_filter.png", "message_filter.png", timeout)
    wait_for_image_and_type_text(dlg, bbox, "message_filter.png", "checked for 4 times", timeout)
    wait_and_click(dlg, bbox, "End_Stage_2.png", "close_opt_console.png", timeout)
    wait_and_click(dlg, bbox, "Truncate_Stage_2.png", "Truncate_Stage_2.png", timeout)
    wait_and_click(dlg, bbox, "Segmentation_complete.png", "Ok_button.png", timeout)
    wait_and_click(dlg, bbox, "Final_dose_calculation.png", "disquete.png", timeout)

except ElementNotFoundError:
    print(f"No se encontró la ventana con '{window_string}' en el título.")


Esperando trigger...
Trigger detectado en (163, 10)
Buscando destino...
Destino encontrado en (163, 10), haciendo clic
Esperando trigger...
Trigger detectado en (954, 571)
Buscando destino...
Destino encontrado en (1044, 632), haciendo clic
Esperando trigger...
Trigger detectado en (163, 10)
Buscando destino...
Destino encontrado en (163, 10), haciendo clic
Esperando trigger...
Trigger detectado en (220, 10)
Buscando destino...
Destino encontrado en (220, 10), haciendo clic
Esperando trigger...
Trigger detectado en (418, 597)
Buscando destino...
Destino encontrado en (418, 597), haciendo clic
Esperando a que aparezca la imagen...
Imagen detectada en (418, 597), escribiendo 'checked for 4 times'
Esperando trigger...
Trigger detectado en (412, 255)
Buscando destino...
Destino encontrado en (766, 145), haciendo clic
Esperando trigger...
Trigger detectado en (880, 61)
Buscando destino...
Destino encontrado en (880, 61), haciendo clic
Esperando trigger...
Trigger detectado en (956, 572)
Bus